# 請結合前面的知識與程式碼，比較不同的 optimizer 與 learning rate 組合對訓練的結果與影響
### 常見的 optimizer 包含
* SGD
* RMSprop
* AdaGrad
* Adam

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
"""
Set your required experiment parameters
"""
LEARNING_RATE = 0.01
EPOCHS = 30
BATCH_SIZE = 512
MOMENTUM = 0.9

In [ ]:
results = {}
"""
建立你的訓練與實驗迴圈並蒐集資料
"""
for name in ['SGD', 'RMSprop', 'AdaGrad', 'Adam']:
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with optimizer:" + name)
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    
    if name == 'SGD':
        optimizer = keras.optimizers.SGD()
    elif name == 'RMSprop':
        optimizer = keras.optimizers.RMSprop()
    elif name == 'AdaGrad':
        optimizer = keras.optimizers.Adagrad()
    else:
        optimizer = keras.optimizers.Adam()
        
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test))
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "exp-optimizer-(name)"
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}

W0717 09:44:33.097256  7340 deprecation_wrapper.py:119] From C:\Users\yjlai\Anaconda3\envs\keras35\lib\site-packages\keras\backend\tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0717 09:44:33.099254  7340 deprecation_wrapper.py:119] From C:\Users\yjlai\Anaconda3\envs\keras35\lib\site-packages\keras\backend\tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0717 09:44:33.115293  7340 deprecation_wrapper.py:119] From C:\Users\yjlai\Anaconda3\envs\keras35\lib\site-packages\keras\backend\tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 09:44:33.116311  7340 deprecation_wrapper.py:119] From C:\Users\yjlai\Anaconda3\envs\keras35\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v

Experiment with optimizer:SGD
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [============================

Epoch 15/30
50000/50000 [==============================] - 2s 46us/step - loss: 1.4626 - acc: 0.4767 - val_loss: 1.5788 - val_acc: 0.4459
Epoch 16/30
50000/50000 [==============================] - 2s 45us/step - loss: 1.4467 - acc: 0.4863 - val_loss: 1.5532 - val_acc: 0.4529
Epoch 17/30
50000/50000 [==============================] - 2s 45us/step - loss: 1.4216 - acc: 0.4900 - val_loss: 1.5375 - val_acc: 0.4450
Epoch 18/30
50000/50000 [==============================] - 2s 46us/step - loss: 1.4005 - acc: 0.4966 - val_loss: 1.5123 - val_acc: 0.4606
Epoch 19/30
50000/50000 [==============================] - 2s 46us/step - loss: 1.3861 - acc: 0.5058 - val_loss: 1.5966 - val_acc: 0.4400
Epoch 20/30
50000/50000 [==============================] - 2s 46us/step - loss: 1.3596 - acc: 0.5153 - val_loss: 1.4691 - val_acc: 0.4808
Epoch 21/30
50000/50000 [==============================] - 2s 45us/step - loss: 1.3431 - acc: 0.5226 - val_loss: 1.5097 - val_acc: 0.4715
Epoch 22/30
50000/50000 [=========

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 3s 54us/step - loss: 2.0323 - acc: 0.2652 - val_loss: 1.8327 - val_acc: 0.3428
Epoch 2/30
50000/50000 [==============================] - 2s 47us/step - loss: 1.7729 - acc: 0.3669 - val_loss: 1.7252 - val_acc: 0.3753
Epoch 3/30
50000/50000 [==============================] - 2s 46us/step - loss: 1.6817 - acc: 0.4004 - val_loss: 1.6324 - val_acc: 0.4201
Epoch 4/30
50000/50000 [==============================] - 2s 46us/step - loss: 1.6172 - acc: 0.4257 - val_loss: 1.6111 - val_acc: 0.4284
Epoch 5/30
50000/50000 [==============================] - 2s 46us/step - loss: 1.5731 - acc: 0.4431 - val_loss: 1.5586 - val_acc: 0.4462
Epoch 6/30
50000/50000 [==============================] - 2s 46us/step - loss: 1.5307 - acc: 0.4570 - val_loss: 1.5241 - val_acc: 0.4648
Epoch 7/30
50000/50000 [==============================] - 2s 46us/step - loss: 1.4889 - acc: 0.4714 - val_loss: 1.4907 - val_acc

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

"""
將實驗結果繪出
"""

color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8, 6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(8, 6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()